In [3]:
import os
import dotenv
import chromadb
from google import genai
from huggingface_hub import InferenceClient

dotenv.load_dotenv()

CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")

HF_MODEL = "sentence-transformers/all-mpnet-base-v2"
HF_TOKEN = os.getenv("HF_TOKEN")

chroma_client = chromadb.HttpClient(host=CHROMA_HOST, port=CHROMA_PORT)
collection = chroma_client.get_or_create_collection("superheroes")

gemini_client = genai.Client()

hf_client = InferenceClient(provider="auto", api_key=HF_TOKEN)

In [4]:
def embed_text(query: str):
    return hf_client.feature_extraction([query], model=HF_MODEL)[0]

In [5]:
def ask_superheroes(question: str, top_k: int = 50) -> str:
    query_vector = embed_text(question)

    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )
    docs = results["documents"][0]
    context = "\n\n".join(docs)

    prompt = (
        "You are Batman's AI. Use the context below to answer the question. "
        "If a name in the question is slightly misspelled or formatted differently, "
        "try to find the closest matching superhero using their name or aliases. "
        "Improvise if needed."
        "If you cannot confidently answer, say 'I don't know'.\n\n"
        "If asked who made you, answer 'I was created by Bruce Wayne using advanced technology and Zur En Arr'"
        "If asked what are your hobbies, answer 'being batman'"
        f"Context:\n{context}\n\n"
        f"Question: {question}\nAnswer:"
    )


    response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [6]:
# 1. Make sure these exist in your script:
# - `collection` → Chroma collection with superhero docs + embeddings
# - `embed_text(text: str)` → returns HF embedding vector
# - `gemini_client` → from google.genai.Client()

# 2. Ask a question
question = "what are your hobbies?"
answer = ask_superheroes(question)

print("Q:", question)
print("A:", answer)

Q: what are your hobbies?
A: being batman
